# Kayak Project

## Imports

In [ ]:
import pandas as pd
import plotly.express as px
import requests
from dotenv import load_dotenv
import os

load_dotenv()
%load_ext dotenv
%dotenv

key = os.getenv('APIKEY')

In [127]:
cities = ["Mont Saint Michel", "St Malo", "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [154]:
df_cities = pd.DataFrame(columns=["city"])
df_cities['city'] = cities
df_cities.reset_index(inplace=True)
df_cities.rename(columns={'index': 'id'}, inplace=True)
df_cities

,id,city
0,0,Mont Saint Michel
1,1,St Malo
2,2,Bayeux
3,3,Le Havre
4,4,Rouen
5,5,Paris
6,6,Amiens
7,7,Lille
8,8,Strasbourg
9,9,Chateau du Haut Koenigsbourg


## Getting API Data

### GPS Coordinates

In [153]:
def get_gps(df):

    df_new = df.copy(deep=True)
    lat_list = []
    lon_list = []
    for i in cities:
        if i == "Gorges du Verdon":
            i = "La%20Palud-sur-Verdon"
            r = requests.get(f"https://nominatim.openstreetmap.org/search?city={i}&format=json").json()
            lat_list.append(r[0]['lat'])
            lon_list.append(r[0]['lon'])
        elif i == 'Ariege':
            r = requests.get(f"https://nominatim.openstreetmap.org/search?county={i}&format=json").json()
            lat_list.append(r[0]['lat'])
            lon_list.append(r[0]['lon'])
        else:
            name = i.replace(" ", "%20")
            r = requests.get(f"https://nominatim.openstreetmap.org/search?city={name}&format=json").json()
            lat_list.append(r[0]['lat'])
            lon_list.append(r[0]['lon'])
        
    df_new['lat'] = lat_list
    df_new['lon'] = lon_list 
    return df_new

In [155]:
df_gps = get_gps(df_cities)
df_gps

,id,city,lat,lon
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,48.649518,-2.0260409
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658
5,5,Paris,48.8588897,2.3200410217200766
6,6,Amiens,49.8941708,2.2956951
7,7,Lille,50.6365654,3.0635282
8,8,Strasbourg,48.584614,7.7507127
9,9,Chateau du Haut Koenigsbourg,48.2495226,7.3454923


### Weather Data

In [149]:
def get_weather(df):
    
    df_new = df.copy(deep=True)
    temps_list = []
    rain_pop = []
    humidity_list = []
    days = list(range(1,8))
    
    for i in df.itertuples():
        lat = i.lat
        lon = i.lon
        r = requests.get(f"https://api.openweathermap.org/data/2.5/onecall?lat={lat}&lon={lon}&units=metric&appid={key}").json()
        weather_7_days = r['daily'][1:]
        temps = [i['feels_like']['day'] for i in weather_7_days]
        rain = [i['pop'] * 100 for i in weather_7_days]
        humidity = [i['humidity'] for i in weather_7_days]
        temps_list.append(temps)
        rain_pop.append(rain)
        humidity_list.append(humidity)
        
    df_new['day_plus'] = [days for _ in range(len(df))]
    df_new['felt_temperatures'] = temps_list
    df_new['rain_chances'] = rain_pop
    df_new['humidity'] = humidity_list
    
    df_new = df_new.set_index(['id']).apply(pd.Series.explode).reset_index()
    df_new[['day_plus', 'felt_temperatures', 'rain_chances', 'humidity']] = df_new[['day_plus', 'felt_temperatures', 'rain_chances', 'humidity']].apply(pd.to_numeric)
    return df_new
    

In [151]:
df_full = get_weather(df_gps)
df_full

,id,city,lat,lon,day_plus,felt_temperatures,rain_chances,humidity
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,1,11.69,36.0,88
1,0,Mont Saint Michel,48.6359541,-1.511459954959514,2,11.47,98.0,81
2,0,Mont Saint Michel,48.6359541,-1.511459954959514,3,7.16,98.0,92
3,0,Mont Saint Michel,48.6359541,-1.511459954959514,4,5.87,100.0,81
4,0,Mont Saint Michel,48.6359541,-1.511459954959514,5,8.88,0.0,50
...,...,...,...,...,...,...,...,...
240,34,La Rochelle,46.1591126,-1.1520434,3,11.90,100.0,90
241,34,La Rochelle,46.1591126,-1.1520434,4,12.61,100.0,86
242,34,La Rochelle,46.1591126,-1.1520434,5,11.61,80.0,85
243,34,La Rochelle,46.1591126,-1.1520434,6,14.75,28.0,73
